In [1]:
!pip install meteor


In [2]:
!pip install evaluate
!pip install rouge_score

  Obtaining dependency information for evaluate from https://files.pythonhosted.org/packages/70/63/7644a1eb7b0297e585a6adec98ed9e575309bb973c33b394dae66bc35c69/evaluate-0.4.1-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24932 sha256=f9d7033bfcd4f393a2d2067343670993cded51eff4c8812151375247e590e1cf
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [3]:
import numpy as np
import glob

In [4]:
# #creating output directory if it doesnt exist
# if not os.path.exists(output_path):
#     os.makedirs(output_path)

In [5]:
!pwd
!ls
%cd ..
!pwd
!ls
%cd input/ensemble-summarizer-module-utilities
!ls
import BART_utilities as bart_u, utilities as norm_u
%cd ../../working
!ls

/kaggle/working
__notebook__.ipynb
/kaggle
/kaggle
input  lib  src  working
/kaggle/input/ensemble-summarizer-module-utilities
BART_utilities.py  utilities.py


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


/kaggle/working
__notebook__.ipynb


In [6]:
def get_summary_data(documents_path, summaries_path):
    all_document_files = glob.glob(documents_path + "/*.txt")
    all_summary_files = glob.glob(summaries_path + "/*.txt")
    
    documents_data = []
    summaries_data = []
    names = []
    
    for filename in all_document_files:
        with open(filename, 'r') as f:
            p = filename.rfind("/")
            names.append(filename[p+1:])
            documents_data.append(f.read())
    
    for filename in all_summary_files:
        with open(filename, 'r') as f: 
            summaries_data.append(f.read())
            
    return names, documents_data, summaries_data

In [7]:
names, documents_data, summaries_data = get_summary_data("/kaggle/input/paper-data/documents", "/kaggle/input/paper-data/summaries")

print(len(names))
print(len(documents_data))
print(len(summaries_data))

100
100
100


In [8]:
#reference_summaries is a list containg all the summaries in the test dataset
reference_summaries = list()
for reference_summary in summaries_data:
    reference_summaries.append(reference_summary)

print(len(reference_summaries))

100


In [9]:
num_tokens_per_summary = list()
document_ids = list()

for i in reference_summaries:
    num_tokens_per_summary.append(len(i.split(" ")))
    
for i in range(1,len(num_tokens_per_summary)+1):
    document_ids.append(i)
    
print(len(num_tokens_per_summary), num_tokens_per_summary[0])
print(len(document_ids))

100 758
100


In [10]:
import torch
from transformers import PegasusForConditionalGeneration, PegasusTokenizer, BartForConditionalGeneration, BartTokenizer
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from transformers import BartTokenizer, BartForConditionalGeneration, AdamW, BartConfig
from BART_utilities import *

#loading gpu
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# Load the Fine-Tuned Pegasus Model
pegasus_tokenizer = AutoTokenizer.from_pretrained("nsi319/legal-pegasus") 
pegasus_model = AutoModelForSeq2SeqLM.from_pretrained("/kaggle/input/output-model")
pegasus_model.to(device)


# Loading Basic Bart Model and Tokenizer
base_bart_model = BartForConditionalGeneration.from_pretrained("facebook/bart-large")
base_bart_model.to(device)

bart_tokenizer = BartTokenizer.from_pretrained('facebook/bart-large', add_prefix_space=True)
new_tokens = ['<F>', '<RLC>', '<A>', '<S>', '<P>', '<R>', '<RPC>']
special_tokens_dict = {'additional_special_tokens': new_tokens}
num_added_toks = bart_tokenizer.add_special_tokens(special_tokens_dict)
base_bart_model.resize_token_embeddings(len(bart_tokenizer))
print(len(bart_tokenizer))


# Load the Fine-Tuned Bart Model
bart_model = bart_u.LitModel(learning_rate = 2e-5, tokenizer = bart_tokenizer, model = base_bart_model)
bart_model.load_state_dict(torch.load("/kaggle/input/bartmodel/outputBARTModel (1).pt"))
bart_model.to(device)

50272


LitModel(
  (model): BartForConditionalGeneration(
    (model): BartModel(
      (shared): Embedding(50272, 1024)
      (encoder): BartEncoder(
        (embed_tokens): Embedding(50272, 1024)
        (embed_positions): BartLearnedPositionalEmbedding(1026, 1024)
        (layers): ModuleList(
          (0-11): 12 x BartEncoderLayer(
            (self_attn): BartAttention(
              (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
            )
            (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
            (activation_fn): GELUActivation()
            (fc1): Linear(in_features=1024, out_features=4096, bias=True)
            (fc2): Linear(in_features=4096, out_features=1024, bias=True)
           

In [11]:
from difflib import SequenceMatcher

def similarity_score(text1, text2):
    return SequenceMatcher(None, text1, text2).ratio()

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def cosine_similarity_score(text1, text2):
    vectorizer = TfidfVectorizer()
    vectors = vectorizer.fit_transform([text1, text2])
    similarity = cosine_similarity(vectors[0], vectors[1])[0][0]
    return similarity

In [13]:
def ensemble_summarizer(text):
    
    # Prepare Legal PEGASUS
    pegasus_input_tokenized = pegasus_tokenizer.encode(text, return_tensors='pt', max_length=1024, truncation=True)
    # Move input tensor to GPU if available
    pegasus_input_tokenized = pegasus_input_tokenized.to(device)
    # Ensure the model and input tensors are on the same device
    with torch.no_grad():
        pegasus_summary_ids = pegasus_model.generate(pegasus_input_tokenized, num_beams=9, no_repeat_ngram_size=3, length_penalty=2.0, 
                                                     min_length=200, max_length=1000, early_stopping=True)
    
     
    # Prepare BART
    bart_input_tokenized = bart_tokenizer.encode(text, truncation=True, return_tensors='pt')
    # Move input tensor to GPU if available
    bart_input_tokenized = bart_input_tokenized.to(device)
    # Ensure the model and input tensors are on the same device
    bart_summary_ids = bart_model.model.to(device).generate(bart_input_tokenized, length_penalty=0.01,
                                                            min_length=200, max_length=1000)
    
      
    # Generate summaries using both
    pegasus_summary = [pegasus_tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in pegasus_summary_ids][0]
    bart_summary = [bart_tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in bart_summary_ids][0]
    

    # Calculate similarity scores
    bart_legal_similarity = cosine_similarity_score(bart_summary, pegasus_summary)
    
    
    # Select the summary with higher similarity or a specific threshold
    # Adjust the threshold as needed
    # Select BART summary if too similar
    # If the similarity is low, choose the longer summary as the ensemble result
    if bart_legal_similarity >= 0.8:
        ensemble_summary = bart_summary
    else:
        if len(bart_summary) >= len(pegasus_summary):
            ensemble_summary = bart_summary
        else:
            ensemble_summary =pegasus_summary
    
    # Return the Ensemble Summary
    return ensemble_summary

In [14]:
# Load the ROUGE metric
from evaluate import load
import evaluate

candidates = list()

for text in documents_data:
    candidates.append(ensemble_summarizer(text))
    
rouge = evaluate.load('rouge')

results = rouge.compute(predictions=candidates, references=reference_summaries)
print(results)

{'rouge1': 0.457718341032395, 'rouge2': 0.2170147858361507, 'rougeL': 0.24366163968876675, 'rougeLsum': 0.3915749212597073}


In [15]:
labelled_docs_df = pd.read_csv("/kaggle/input/summary-dataset-rr-labelled-documents/labeled_docs.csv")
labelled_docs_df.head(5)

,i,pred,emb,doc
0,11,"[4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, ...","[[0.27125, 0.43958, -0.40032, -0.59358, 1.0063...","['(Civil) No. 548 of 1987.', '(Under Article 3..."
1,13,"[3, 3, 4, 4, 4, 4, 8, 8, 8, 8, 8, 5, 4, 5, 5, ...","[[-0.99177, 0.18211, -0.47092, 0.25772, 0.4945...","['N: Criminal Appeal Nos.', '287 & 288 of 1980..."
2,63,"[4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, ...","[[-0.49523, 0.00739, -0.61337, -0.23871, 0.733...","[': Criminal Appeal No 133 of 1975.', 'Appeal ..."
3,82,"[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 5, 5, 5, 4, ...","[[0.14106, 0.0147, -0.92286, -0.40899, 0.63139...","['Appeal No. 1951 of 1975.', 'Appeal by Specia..."
4,18,"[3, 4, 4, 4, 4, 8, 5, 5, 5, 5, 5, 5, 5, 5, 5, ...","[[-0.03027, 0.1304, -0.67327, -0.23739, 0.4874...","['Civil Appeal No. 598 of 1980.', 'Appeal by S..."


In [16]:
import json

roles = ['PREAMBLE','NONE','FAC','ISSUE','ARG_PETITIONER','RLC','ANALYSIS', 'PRE_RELIED','RATIO','RPC','STA','ARG_RESPONDENT','PRE_NOT_RELIED']

# Read the saved tag2idx from file
with open('/kaggle/input/rr-tags-to-idx-mapping/tag2idx.json', 'r') as file:
    tag2idx = json.load(file)
    idx2tag = {v: k for k, v in tag2idx.items()}

In [17]:
# # a new function that takes in the list of rhetorical roles and the judgements, 
# # removes each rhetorical role one by one and generates the summary using ensemble summarizer. and stores that summary.
# # so we will need 13 lists each list consisting of the summaries of all the documents formed after removing that particular rhetorical role.
# import ast
# from nltk.translate.bleu_score import corpus_bleu, SmoothingFunction

# rogueResults = dict()
# bleu1Results = dict()
# bleu2Results = dict()

# def modify_document(role, doc_labels, doc_texts):
#     updated_text = []
#     for i, label in enumerate(doc_labels):
#         if idx2tag[label] == role:
#             updated_text.append(doc_texts[i])
#     return ' '.join(updated_text)


# def remove_rhetorical_role(documents_data, role, labelled_docs_df):
#     # Initialize new return dataframe
#     updated_documents_df = pd.DataFrame(columns=['i', 'doc_text'])
    
#     # For each of the labelled docs, modify the text
#     for i, row in labelled_docs_df.iterrows():
#         doc_num, doc_texts, doc_labels = row['i'], ast.literal_eval(row['doc']), ast.literal_eval(row['pred'])
#         updated_text = modify_document(role, doc_labels, doc_texts)
#         updated_documents_df.loc[len(updated_documents_df)] = [doc_num, updated_text]

#     # Return a list of all the updated document texts
#     return updated_documents_df['doc_text'].tolist()


# def rhetorical_role_analysis(documents_data, roles, labelled_docs_df):
#     for role in roles:
#         rr_removed_summaries = list()
#         updated_documents = remove_rhetorical_role(documents_data, role, labelled_docs_df)

#         for document_text in updated_documents:
#             rr_removed_summaries.append(ensemble_summarizer(document_text))
            
# #         rouge = evaluate.load('rouge')

# #         results_references = rouge.compute(predictions=rr_removed_summaries, references=reference_summaries)
# #         results_generated = rouge.compute(predictions=rr_removed_summaries, references=candidates)
        
# #         rogueResults[role] = dict()
# #         rogueResults[role]["Reference"] = results_references
# #         rogueResults[role]["Generated"] = results_generated
        
        
# #         bleu1Results[role] = dict()
# #         bleu1Results[role]["Reference"]= corpus_bleu(reference_summaries, rr_removed_summaries)
# #         bleu1Results[role]["Generated"]= corpus_bleu(candidates, rr_removed_summaries)
        
#         bleu2Results[role] = dict()
#         bleu2Results[role]["Reference"] = corpus_bleu(reference_summaries, rr_removed_summaries, weights=(0.5, 0.5), smoothing_function=SmoothingFunction().method4)
#         bleu2Results[role]["Generated"] = corpus_bleu(candidates, rr_removed_summaries, weights=(0.5, 0.5), smoothing_function=SmoothingFunction().method4)
        

In [18]:
# rhetorical_role_analysis(documents_data, roles, labelled_docs_df)
# # print(rogueResults)

In [19]:
# print(bleu1Results)
# with open("/kaggle/working/bleu1Results.json", 'w') as jf:
#     json.dump(bleu1Results, jf)

In [20]:
# print(bleu2Results)
# with open("/kaggle/working/bleu2Results.json", 'w') as jf:
#     json.dump(bleu2Results, jf)

In [21]:
# with open("/kaggle/working/AllRogueResults.json", 'w') as jf:
#     json.dump(rogueResults, jf)

In [22]:
import json
import ast
import pandas as pd
from nltk.translate.bleu_score import corpus_bleu, SmoothingFunction

# Initialize dictionaries for storing results
rogueResults = dict()
bleu1Results = dict()
bleu2Results = dict()

# Modify document function to include 'FAC' and 'ANALYSIS' roles
def modify_document(roles, doc_labels, doc_texts):
    updated_text = []
    for i, label in enumerate(doc_labels):
        if idx2tag[label] in roles:
            updated_text.append(doc_texts[i])
    return ' '.join(updated_text)



# Function for rhetorical role analysis
def rhetorical_role_analysis(documents_data, roles, labelled_docs_df):
    rouge = evaluate.load('rouge')
    
    for role in roles:
        rr_removed_summaries = []
        for _, row in labelled_docs_df.iterrows():
            doc_texts, doc_labels = ast.literal_eval(row['doc']), ast.literal_eval(row['pred'])
            updated_text = modify_document([role], doc_labels, doc_texts)
            rr_removed_summaries.append(ensemble_summarizer(updated_text))
        
        rogueResults[role] = rouge.compute(predictions=rr_removed_summaries, references=reference_summaries)
        
        bleu1Results[role] = dict()
        bleu1Results[role]["Reference"] = corpus_bleu(reference_summaries, rr_removed_summaries)
        bleu1Results[role]["Generated"] = corpus_bleu(candidates, rr_removed_summaries)
        
#         bleu2Results[role] = dict()
#         bleu2Results[role]["Reference"] = corpus_bleu(reference_summaries, rr_removed_summaries, weights=(0.5, 0.5), smoothing_function=SmoothingFunction().method4)
#         bleu2Results[role]["Generated"] = corpus_bleu(candidates, rr_removed_summaries, weights=(0.5, 0.5), smoothing_function=SmoothingFunction().method4)

    # For FAC and ANALYSIS combined
    combined_roles = ['FAC', 'ANALYSIS']
    combined_summaries = []
    for _, row in labelled_docs_df.iterrows():
        doc_texts, doc_labels = ast.literal_eval(row['doc']), ast.literal_eval(row['pred'])
        updated_text = modify_document(combined_roles, doc_labels, doc_texts)
        combined_summaries.append(ensemble_summarizer(updated_text))
    
    rogueResults['FAC+ANALYSIS'] = rouge.compute(predictions=combined_summaries, references=reference_summaries)
    
    bleu1Results['FAC+ANALYSIS'] = dict()
    bleu1Results['FAC+ANALYSIS']["Reference"] = corpus_bleu(reference_summaries, combined_summaries)
    bleu1Results['FAC+ANALYSIS']["Generated"] = corpus_bleu(candidates, combined_summaries)
    
#     bleu2Results['FAC+ANALYSIS'] = dict()
#     bleu2Results['FAC+ANALYSIS']["Reference"] = corpus_bleu(reference_summaries, combined_summaries, weights=(0.5, 0.5), smoothing_function=SmoothingFunction().method4)
#     bleu2Results['FAC+ANALYSIS']["Generated"] = corpus_bleu(candidates, combined_summaries, weights=(0.5, 0.5), smoothing_function=SmoothingFunction().method4)

# Perform rhetorical role analysis
rhetorical_role_analysis(documents_data, roles, labelled_docs_df)

# Print and save results
print(rogueResults)
with open("/kaggle/working/rogueResults.json", 'w') as jf1:
    json.dump(rogueResults, jf1)

print(bleu1Results)
with open("/kaggle/working/bleu1Results.json", 'w') as jf2:
    json.dump(bleu1Results, jf2)

print(bleu2Results)
with open("/kaggle/working/bleu2Results.json", 'w') as jf3:
    json.dump(bleu2Results, jf3)


/opt/conda/lib/python3.10/site-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


{'PREAMBLE': {'rouge1': 0.2126147710914883, 'rouge2': 0.04782074249732568, 'rougeL': 0.13059565867134, 'rougeLsum': 0.19095077263710142}, 'NONE': {'rouge1': 0.2316215608800694, 'rouge2': 0.05629813729427509, 'rougeL': 0.1402060823493546, 'rougeLsum': 0.20606397967510434}, 'FAC': {'rouge1': 0.31219062615501425, 'rouge2': 0.06830368868990969, 'rougeL': 0.16182368357659999, 'rougeLsum': 0.27061456313320414}, 'ISSUE': {'rouge1': 0.13540685240108793, 'rouge2': 0.0277947747829259, 'rougeL': 0.09531151686690859, 'rougeLsum': 0.12453095343565468}, 'ARG_PETITIONER': {'rouge1': 0.18046815329015423, 'rouge2': 0.03959909025578672, 'rougeL': 0.1137230125781096, 'rougeLsum': 0.16296343287289458}, 'RLC': {'rouge1': 0.2099459342525096, 'rouge2': 0.048786254224635915, 'rougeL': 0.1253383223179695, 'rougeLsum': 0.1870144304920141}, 'ANALYSIS': {'rouge1': 0.30532535102338465, 'rouge2': 0.06562001912684173, 'rougeL': 0.15782453510047167, 'rougeLsum': 0.2640169500087824}, 'PRE_RELIED': {'rouge1': 0.1910306